In [92]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [93]:
a=np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b=np.diag(a)
print(b)
np.diag(b)  # convert back to NxN

[1 5 9]


array([[1, 0, 0],
       [0, 5, 0],
       [0, 0, 9]])

In [94]:
class QuboPoly():
    def __init__(self, n=1024):
        self.array=np.zeros((n, n), dtype=int)
        self.constant=0
        self._size=n
    
    def add_term(self, i, j, c):
        if i>=self._size or j>=self._size:
            raise RuntimeError("Wrong index")
        self.array[i][j]+=c
        
    def add_constant(self, c):
        self.constant+=c
        
    def sum(self, p):
        if self._size != p._size:
            raise RuntimeError("Wrong polynomial size")
        self.array+=p.array
        self.constant+=p.constant
        
    def power(self):
        a=np.diag(self.array)
        self.array=np.outer(a, a) + 2*self.constant*np.diag(a) # convert back to NxN
        self.constant**=2
        
    def multiply(self, p):
        a=np.diag(self.array)
        b=np.diag(p.array)
        self.array=np.outer(a, b) + self.constant*np.diag(b) + p.constant*np.diag(a)
        self.constant*=p.constant

In [106]:
N=4
grid=np.arange(N*N).reshape(N, N)
print(grid)
grid[:, 0]  # select column
grid[:, 1]
grid[:, 2]

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


array([ 2,  6, 10, 14])

#### Sum vertical for each column (vertical one-hot)

In [96]:
def build_column_penalty(N):
    qubo=QuboPoly(N*N)
    for col in range(N):
        tmp=QuboPoly(N*N)
        for i in grid[:, col]:
            tmp.add_term(i, i, 1)
        tmp.add_constant(-1)
        tmp.power()
        qubo.sum(tmp)
    return qubo

#### Sum horizontal for each row (horizontal one-hot)

In [97]:
def build_row_penalty(N):
    qubo=QuboPoly(N*N)
    for row in range(N):
        tmp=QuboPoly(N*N)
        for i in grid[row]:
            tmp.add_term(i, i, 1)
        tmp.add_constant(-1)
        tmp.power()
        qubo.sum(tmp)
    return qubo

#### Sum diagonal for each diagonal

In [98]:
np.diag(grid, k=-1)

array([ 8, 17, 26, 35, 44, 53, 62])

In [99]:
np.fliplr(grid)

array([[ 7,  6,  5,  4,  3,  2,  1,  0],
       [15, 14, 13, 12, 11, 10,  9,  8],
       [23, 22, 21, 20, 19, 18, 17, 16],
       [31, 30, 29, 28, 27, 26, 25, 24],
       [39, 38, 37, 36, 35, 34, 33, 32],
       [47, 46, 45, 44, 43, 42, 41, 40],
       [55, 54, 53, 52, 51, 50, 49, 48],
       [63, 62, 61, 60, 59, 58, 57, 56]])

In [100]:
def build_diag_penalty(N):
    qubo=QuboPoly(N*N)
    for g in [grid, np.fliplr(grid)]:
        for k in range(-N+2, N-1):
            tmp1=QuboPoly(N*N)
            tmp2=QuboPoly(N*N)
            for dia in np.diag(g, k=k):
                tmp1.add_term(dia, dia, 1)
                tmp2.add_term(dia, dia, 1)
            tmp1.add_constant(-1)
            tmp1.multiply(tmp2)
            qubo.sum(tmp1)
    return qubo

#### build QUBO matrix

In [135]:
N=8
grid=np.arange(N*N).reshape(N, N)

P1=build_column_penalty(N)
P2=build_row_penalty(N)
P3=build_diag_penalty(N)

qubo_obj=QuboPoly(N*N)
qubo_obj.sum(P1)
qubo_obj.sum(P2)
qubo_obj.sum(P3)

#print(qubo_obj.array)
#print(qubo_obj.constant)

In [136]:
m=gp.Model()

x=m.addMVar(N*N, vtype=GRB.BINARY)

m.setObjective(x@qubo_obj.array@x + qubo_obj.constant, GRB.MINIMIZE)
m.write('debug.lp')

In [137]:
m.optimize()
m.write('result.sol')

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 0 rows, 64 columns and 0 nonzeros
Model fingerprint: 0x7700af9d
Model has 792 quadratic objective terms
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 16.0000000
Found heuristic solution: objective 4.0000000
Found heuristic solution: objective 2.0000000
Presolve time: 0.02s
Presolved: 728 rows, 792 columns, 2184 nonzeros
Variable types: 0 continuous, 792 integer (792 binary)

Root relaxation: objective -4.800000e+01, 66 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bo

In [138]:
solution=np.zeros(N*N, dtype=int)
for index, var in enumerate(m.getVars()):
    solution[index]=int(var.x)
    
print(solution)

[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]


In [139]:
grid_sol=solution.reshape(N, N)
print(grid_sol)

[[0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0]]
